# Tracing Lsass Memory Dumping with Outflank-Dumpert (via syscall)

### Dataset Description
Below dataset shows the adversaries technique to dump memory content of lsass using system calls (syscalls) and API unhooking.
https://raw.githubusercontent.com/OTRF/Security-Datasets/master/datasets/atomic/windows/credential_access/host/cmd_lsass_memory_dumpert_syscalls.zip

### Adversary View
```
C:\Users\wardog\Desktop>Outflank-Dumpert.exe
________          __    _____.__                 __
\_____  \  __ ___/  |__/ ____\  | _____    ____ |  | __
  /   |   \|  |  \   __\   __\|  | \__  \  /    \|  |/ /
/    |    \  |  /|  |  |  |  |  |__/ __ \|   |  \    <
\_______  /____/ |__|  |__|  |____(____  /___|  /__|_ \
        \/                             \/     \/     \/
                                  Dumpert
                              By Cneeliz @Outflank 2019

[1] Checking OS version details:
        [+] Operating System is Windows 10 or Server 2016, build number 18363
        [+] Mapping version specific System calls.
[2] Checking Process details:
        [+] Process ID of lsass.exe is: 756
        [+] NtReadVirtualMemory function pointer at: 0x00007FFB929DC890
        [+] NtReadVirtualMemory System call nr is: 0x3f
        [+] Unhooking NtReadVirtualMemory.
[3] Create memorydump file:
        [+] Open a process handle.
        [+] Dump lsass.exe memory to: \??\C:\windows\Temp\dumpert.dmp
        [+] Dump succesful.

C:\Users\wardog\Desktop>
```

### Tactics, Techniques
Tactics: [TA0006](https://attack.mitre.org/tactics/TA0006)
Techniques: [T1003.001](https://attack.mitre.org/tactics/TA0006)

The tool targets lsass.exe (Local Security Authority Subsystem Service), which handles password authentication in Windows. 

We can start by looking at the process executed by this program:

In [1]:
procs = get process from
            https://raw.githubusercontent.com/OTRF/Security-Datasets/master/datasets/atomic/windows/credential_access/host/cmd_lsass_memory_dumpert_syscalls.zip
            where command_line LIKE '%Dump%'

VARIABLE,TYPE,#(ENTITIES),#(RECORDS),artifact*,directory*,file*,process*,user-account*,x-oca-asset*,x-oca-event*
procs,process,1,23,24,46,47,26,3,24,24


Examine the process:

In [2]:
disp procs
proc = procs

pid 
 name 
 id 
 command_line 
 x_unique_id 
 created 
 binary_ref.name 
 binary_ref.id 
 binary_ref.hashes.'SHA-1' 
 binary_ref.hashes.MD5 
 binary_ref.hashes.'SHA-256' 
 binary_ref.parent_directory_ref.path 
 binary_ref.parent_directory_ref.id 
 parent_ref.pid 
 parent_ref.name 
 parent_ref.id 
 parent_ref.command_line 
 parent_ref.x_unique_id 
 parent_ref.created 
 parent_ref.parent_ref.pid 
 parent_ref.parent_ref.name 
 parent_ref.parent_ref.id 
 parent_ref.parent_ref.command_line 
 parent_ref.parent_ref.x_unique_id 
 parent_ref.parent_ref.created 
 parent_ref.binary_ref.name 
 parent_ref.binary_ref.id 
 parent_ref.binary_ref.hashes.'SHA-1' 
 parent_ref.binary_ref.hashes.MD5 
 parent_ref.binary_ref.hashes.'SHA-256' 
 parent_ref.binary_ref.parent_directory_ref.path 
 parent_ref.binary_ref.parent_directory_ref.id 
 parent_ref.creator_user_ref.user_id 
 parent_ref.creator_user_ref.id 
 creator_user_ref.user_id 
 creator_user_ref.id 
 
 
 
 
 6772 
 Outflank-Dumpert.exe 
 process--a61744bb-b1ca-565e-a300-7fe978fc9ad2 
 Outflank-Dumpert.exe 
 {39e4a257-004e-5f8d-4304-000000000700} 
 2020-10-19T02:56:14.283Z 
 Outflank-Dumpert.exe 
 file--502dbda9-3a89-53e7-99a4-507a3df022a7 
 98B13ABB701577AC88E360D702BB89160B50E378 
 C78D58386504CAF3437EED37FBFF77B5 
 D286348DE31501385A3A8F57A08AF602722913C36AAA4AA9FCB9EE129A7680C0 
 C:\Users\wardog\Desktop 
 directory--a0157638-19ed-5309-a828-eb616733ae0e 
 3080 
 cmd.exe 
 process--6c2c870b-d62a-5abc-ac05-0b391d932529 
 "C:\windows\system32\cmd.exe" 
 {39e4a257-ff60-5f8c-2f04-000000000700} 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 cmd.exe 
 file--bdb40b46-65f7-5a74-9516-60746576d4e7 
 None 
 None 
 None 
 C:\Windows\System32 
 directory--0a58d0c1-59e6-5afd-8252-dcd3f13e5622 
 None 
 None 
 WORKSTATION5\wardog 
 user-account--c8b6cbe8-e16c-5b03-948a-085d1333e43e 
 
 
 
 Block Executed in 1 seconds 
 
 
 VARIABLE 
 TYPE 
 #(ENTITIES) 
 #(RECORDS) 
 artifact* 
 directory* 
 file* 
 process* 
 user-account* 
 x-oca-asset* 
 x-oca-event* 
 
 
 
 
 proc 
 process 
 1 
 23 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 *Number of related records cached.

We found one process; let's examine its parent process

In [3]:
parents = find process created proc
disp parents

pid 
 name 
 id 
 command_line 
 x_unique_id 
 created 
 binary_ref.name 
 binary_ref.id 
 binary_ref.hashes.'SHA-1' 
 binary_ref.hashes.MD5 
 binary_ref.hashes.'SHA-256' 
 binary_ref.parent_directory_ref.path 
 binary_ref.parent_directory_ref.id 
 parent_ref.pid 
 parent_ref.name 
 parent_ref.id 
 parent_ref.command_line 
 parent_ref.x_unique_id 
 parent_ref.created 
 parent_ref.parent_ref.pid 
 parent_ref.parent_ref.name 
 parent_ref.parent_ref.id 
 parent_ref.parent_ref.command_line 
 parent_ref.parent_ref.x_unique_id 
 parent_ref.parent_ref.created 
 parent_ref.binary_ref.name 
 parent_ref.binary_ref.id 
 parent_ref.binary_ref.hashes.'SHA-1' 
 parent_ref.binary_ref.hashes.MD5 
 parent_ref.binary_ref.hashes.'SHA-256' 
 parent_ref.binary_ref.parent_directory_ref.path 
 parent_ref.binary_ref.parent_directory_ref.id 
 parent_ref.creator_user_ref.user_id 
 parent_ref.creator_user_ref.id 
 creator_user_ref.user_id 
 creator_user_ref.id 
 
 
 
 
 3080 
 cmd.exe 
 process--6c2c870b-d62a-5abc-ac05-0b391d932529 
 "C:\windows\system32\cmd.exe" 
 {39e4a257-ff60-5f8c-2f04-000000000700} 
 None 
 cmd.exe 
 file--bdb40b46-65f7-5a74-9516-60746576d4e7 
 None 
 None 
 None 
 C:\Windows\System32 
 directory--0a58d0c1-59e6-5afd-8252-dcd3f13e5622 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 
 
 
 Block Executed in 2 seconds 
 
 
 VARIABLE 
 TYPE 
 #(ENTITIES) 
 #(RECORDS) 
 artifact* 
 directory* 
 file* 
 process* 
 user-account* 
 x-oca-asset* 
 x-oca-event* 
 
 
 
 
 parents 
 process 
 1 
 5 
 27 
 50 
 53 
 30 
 4 
 27 
 27 
 
 
 *Number of related records cached.

It's executed by `cmd.exe`, let's check to whom this parent process belongs

In [4]:
users = find user-account owned procs

VARIABLE,TYPE,#(ENTITIES),#(RECORDS),artifact*,directory*,file*,process*,user-account*,x-oca-asset*,x-oca-event*
users,user-account,1,4,28,52,55,33,4,28,28


In [5]:
disp users

user_id,id
WORKSTATION5\wardog,user-account--c8b6cbe8-e16c-5b03-948a-085d1333e43e


It appears that `WORKSTATION5\wardog` is the executor. We can gather further information, such as the hostname and Sysmon events, which will be useful for the incident report.

In [6]:
hosts = find x-oca-asset linked procs
disp hosts
events = find x-oca-event linked procs
disp events

hostname 
 id 
 
 
 
 
 WORKSTATION5 
 x-oca-asset--40bf1119-f018-56a0-a6a5-9bfeab2bc1df 
 
 
 
 
 
 
 module 
 provider 
 code 
 action 
 id 
 
 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 1 
 Process Create 
 x-oca-event--df474eaf-9f24-4ad4-9a44-95e6f081cb43 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 5 
 Process terminated 
 x-oca-event--35a316a1-e236-45df-be63-a0fb3f9988cd 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 7 
 Image loaded 
 x-oca-event--e8745538-6119-40d2-8c5c-2a02be6ec132 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 7 
 Image loaded 
 x-oca-event--e09f65fd-c493-4b07-8c74-da0a06bc8020 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 7 
 Image loaded 
 x-oca-event--deb6872f-4e26-4a80-b1da-b25ce27146b1 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 11 
 File created 
 x-oca-event--bce96601-bb41-48e5-aa46-24e2d57cd2d7 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 7 
 Image loaded 
 x-oca-event--cd1a646a-2270-4769-a292-cc2f9e964dcf 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 7 
 Image loaded 
 x-oca-event--5bd9a784-38c6-46e6-bbd7-eaed882a6ac1 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 7 
 Image loaded 
 x-oca-event--fcb4a9e7-30fc-4a35-a7cd-752ddece5098 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 7 
 Image loaded 
 x-oca-event--7bfcee3c-426b-4b8b-975e-978c599abbf5 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 7 
 Image loaded 
 x-oca-event--86051a43-913d-4cbc-83b7-de3eb5e0da5a 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 7 
 Image loaded 
 x-oca-event--ff846fe2-46ef-4a8b-b480-e32237df75a4 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 7 
 Image loaded 
 x-oca-event--99ab71b6-0c8e-41b6-8a12-2930477235bc 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 7 
 Image loaded 
 x-oca-event--0d7be2ef-a3df-4cc2-96b3-dccbeed779d2 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 7 
 Image loaded 
 x-oca-event--655cc13a-3928-4f2c-afde-059984908db1 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 7 
 Image loaded 
 x-oca-event--6c2a4f07-c3d8-434b-8c04-4fdb7341b3cf 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 7 
 Image loaded 
 x-oca-event--0c0aacea-3dfe-4506-b140-6ade68ee21fa 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 7 
 Image loaded 
 x-oca-event--50c6120f-61e3-4f91-b5fb-23e8b78f6a51 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 7 
 Image loaded 
 x-oca-event--e66e95f1-ac47-4db1-be92-38906eda9b04 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 7 
 Image loaded 
 x-oca-event--92c6f257-bd76-4e6a-9846-0ca01b28be3a 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 7 
 Image loaded 
 x-oca-event--bc33e995-9587-45f1-b177-7eb8b7710838 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 7 
 Image loaded 
 x-oca-event--9186dafb-b3ab-413c-94da-534520816c5e 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 1 
 Process Create 
 x-oca-event--6c695295-4bf8-4221-8b7c-d8b5c1bdae52 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 1 
 Process Create 
 x-oca-event--33a9b694-10db-4c41-9828-dceb507c0b16 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 1 
 Process Create 
 x-oca-event--9571c0b7-10f0-4fbb-ad2b-acd6b2f22926 
 
 
 
 Block Executed in 1 seconds 
 
 
 VARIABLE 
 TYPE 
 #(ENTITIES) 
 #(RECORDS) 
 artifact* 
 directory* 
 file* 
 process* 
 user-account* 
 x-oca-asset* 
 x-oca-event* 
 
 
 
 
 hosts 
 x-oca-asset 
 1 
 28 
 28 
 52 
 55 
 33 
 5 
 27 
 28 
 
 
 events 
 x-oca-event 
 25 
 25 
 28 
 52 
 55 
 33 
 5 
 28 